In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as optimization
from scipy.optimize import least_squares
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

In [2]:
# path1 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero/zero_csv/ant1/zero_csvdf_ant1_zero_7.csv'
# path2 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero/zero_csv/ant2/zero_csvdf_ant2_zero_7.csv'
# path3 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero/zero_csv/ant3/zero_csvdf_ant3_zero_7.csv'
path = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero/zero_csv/no_sep_IQ_info_zero_1.csv'
# path1 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/45degrees/df_ant1_45_2.csv'
# path2 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/45degrees/df_ant2_45_2.csv'
# path3 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/45degrees/df_ant3_45_2.csv'
# path1 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/30degrees/df_ant1_30_2.csv'
# path2 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/30degrees/df_ant2_30_2.csv'
# path3 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/30degrees/df_ant3_30_2.csv'
# path1 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero_degree_zero_posi/df_ant1_0_posi0_2.csv'
# path2 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero_degree_zero_posi/df_ant2_0_posi0_2.csv'
# path3 = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zero_degree_zero_posi/df_ant3_0_posi0_2.csv'

# df1 = pd.read_csv(path1) # in each channel we have 96 data points per antenna
# df2 = pd.read_csv(path2)
# df3 = pd.read_csv(path3)
df1 = pd.read_csv(path)
len(df1)

19536

# Correction_ optimized
Here we calculate the results that the corrections are optimized. It means that even the correction files are optimized first for calculations.  
Yet, the final result shows that the optimized correction actually does not improve the precision of angle estimation here. As a result, we will ignore this optimizatopn. 

In [3]:
t = df1[df1.Channel >= 11]
t['omega'] = (t['Channel']-11)*2+ 2428.25
tless = df1[df1.Channel < 11]
tless['omega'] = (tless['Channel'])*2 + 2404.25
df1 = t.append(tless)

<ipython-input-3-3d26178556e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['omega'] = (t['Channel']-11)*2+ 2428.25
<ipython-input-3-3d26178556e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tless['omega'] = (tless['Channel'])*2 + 2404.25


# Optimize the points
Using non-linear least squre fitting. to fit the pounts to the expected curve. 

In [4]:
df1['ts'] = df1['time_stamp']
# df = df1.drop(df1[df1.time_per_channel > 151.75].index)

In [5]:
ts = np.arange(0.25,74.25, 0.25)
N = int(len(df1)/296)
for n in range(N):
    for i in range(len(ts)):
        df1['ts'][i+n*296] = ts[i]
# df1[(N-1)*296:296*(N)] 

<ipython-input-5-3383be4a4c02>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ts'][i+n*296] = ts[i]


In [6]:
# df1.unstack(level=0)[['ts']].plot(subplots=True, title="ts", color= '#28156c', label='ts', legend=True)

In [8]:
# optomize the data from ANT1
def s(theta, t ):
    x = theta[0] * np.cos(2*np.pi*t*fc + phi)
    y = theta[1] * np.sin(2*np.pi*t*fc + phi)
    return np.array([x, y])

# ts = df1.ts
new_i_lst = []
new_q_lst = []

phi = 0;
for i in range(0, len(df1), 296):
    Ai = df1.Magnitude[i:i+296].mean()
    Aq = df1.Magnitude[i:i+296].mean()
    fc = df1.omega[i:i+296].mean(); 
    ts = df1.time_stamp[i:i+296]
    ss = s([Ai, Aq, fc, phi], ts)
    ss[0] += df1['I'][i:i+296]
    ss[1] += df1['Q'][i:i+296]
    

    def fun(theta):
        return (ss- s(theta, ts)).flatten()

    theta0 = [0,0,0,0]
    res1 = least_squares(fun, theta0)
    N = int(len(res1.fun)/2)
    new_i1 = res1.fun[0:N]
    new_i_lst.append(new_i1)
    new_q1 = res1.fun[N:len(res1.fun)]
    new_q_lst.append(new_q1)

In [13]:
df1['time_per_channel'] =  df1['time_stamp']
for i in range(0, len(df1)- 296, 296):
    df1['time_per_channel'][i+296:i+2*296] = df1['time_stamp'][0:296]

<ipython-input-13-6c7b455bee49>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['time_per_channel'][i+296:i+2*296] = df1['time_stamp'][0:296]


In [14]:
from itertools import chain
flatten_i = list(chain.from_iterable(new_i_lst))
flatten_q = list(chain.from_iterable(new_q_lst))
df1['new_i'] = flatten_i
df1['new_q'] = flatten_q
df1[0:288]

,pkt,Channel,ant_info,I,Q,Magnitude,time_stamp,sample_idx,omega,ts,time_per_channel,new_i,new_q
0,0,35,(65+7j),65,7,65.375837,9.00,36,2476.25,0.25,9.00,65.000000,10.478436
1,0,35,(62-22j),62,-22,65.787537,9.25,37,2476.25,0.50,9.25,65.500971,-18.786344
2,0,35,(52-45j),52,-45,68.767725,9.50,38,2476.25,0.75,9.50,58.468952,-42.540374
3,0,35,(33-65j),33,-65,72.897188,9.75,39,2476.25,1.00,9.75,41.452093,-63.668860
4,0,35,(-30+46j),-30,46,54.918121,11.00,44,2476.25,1.25,11.00,-30.000000,42.521564
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,35,(18+48j),18,48,51.264022,149.75,599,2476.25,71.00,149.75,26.452093,49.331140
284,0,35,(-6-25j),-6,-25,25.709920,151.00,604,2476.25,71.25,151.00,-6.000000,-28.478436
285,0,35,(-13-22j),-13,-22,25.553865,151.25,605,2476.25,71.50,151.25,-16.500971,-25.213656
286,0,35,(-19-13j),-19,-13,23.021729,151.50,606,2476.25,71.75,151.50,-25.468952,-15.459626


In [15]:
df = df1.drop(df1[df1.time_per_channel > 151.75].index)
N = int(len(df)/288)

In [16]:
lst_ant1 = []
lst_ant2 = []
lst_ant3 = []

for i in range(0, len(df), 12):    
    lst_ant2.append(df[i:i+4])
    lst_ant1.append(df[i+4:i+8])
    lst_ant3.append(df[i+8:i+12])

In [18]:
df_ant1 = pd.concat(lst_ant1,  ignore_index=True)
df_ant2 = pd.concat(lst_ant2, ignore_index=True)
df_ant3 = pd.concat(lst_ant3, ignore_index=True)

In [22]:
X12 = (df_ant1['I'] + 1j*df_ant1['Q']) * (df_ant2['I'] - 1j*df_ant2['Q'])
X23 = (df_ant2['I'] + 1j*df_ant2['Q']) * (df_ant3['I'] - 1j*df_ant3['Q'])
X13 = (df_ant1['I'] + 1j*df_ant1['Q']) * (df_ant3['I'] - 1j*df_ant3['Q'])
X12_opt = (df_ant1['new_i'] + 1j*df_ant1['new_q']) * (df_ant2['new_i'] - 1j*df_ant2['new_q'])
X23_opt = (df_ant2['new_i'] + 1j*df_ant2['new_q']) * (df_ant3['new_i'] - 1j*df_ant3['new_q'])
X13_opt = (df_ant1['new_i'] + 1j*df_ant1['new_q']) * (df_ant3['new_i'] - 1j*df_ant3['new_q'])

In [23]:
d = {'channel': df_ant1['Channel'],'omega':df_ant1['omega']}
data = pd.DataFrame(data=d)
data['phi12'] = np.angle(X12)
data['phi23'] = np.angle(X23)
data['phi13'] = np.angle(X13)
data['mag12'] = np.abs(X12)
data['mag23'] = np.abs(X23)
data['mag13'] = np.abs(X13)

In [24]:
data['phi12_opt'] = np.angle(X12_opt)
data['phi23_opt'] = np.angle(X23_opt)
data['phi13_opt'] = np.angle(X13_opt)
data['mag12_opt'] = np.abs(X12_opt)
data['mag23_opt'] = np.abs(X23_opt)
data['mag13_opt'] = np.abs(X13_opt)

In [25]:
phi_channel = []
for i in range(0, len(data),96):
    phi_channel.append({"channel":data['channel'][i], "omega":data['omega'][i],
                       "ave_phi12": np.average(data['phi12'][i:i+96]),
                       "ave_phi23": np.average(data['phi23'][i:i+96]),
                        "ave_phi13": np.average(data['phi13'][i:i+96]),
                        "ave_phi12_opt": np.average(data['phi12_opt'][i:i+96]),
                       "ave_phi23_opt": np.average(data['phi23_opt'][i:i+96]),
                        "ave_phi13_opt": np.average(data['phi13_opt'][i:i+96]),
                       "ave_mag12": np.average(data['mag12'][i:i+96]),
                       "ave_mag23": np.average(data['mag23'][i:i+96]),
                       "ave_mag13": np.average(data['mag13'][i:i+96]),})
average_perchannel = pd.DataFrame(phi_channel)

In [26]:
print(' angle per paire', np.average(average_perchannel['ave_phi12'])*180/np.pi, np.average(average_perchannel['ave_phi23'])*180/np.pi, np.average(average_perchannel['ave_phi13'])*180/np.pi,
     '\n average mag per paire', np.average(average_perchannel['ave_mag12']), np.average(average_perchannel['ave_mag23']), np.average(average_perchannel['ave_mag13']),
     '\n total average of angle', (np.average(average_perchannel['ave_phi12'])/3+ np.average(average_perchannel['ave_phi23'])/3+ np.average(average_perchannel['ave_phi13'])/3)*180/np.pi,
     '\n STD per paire', average_perchannel['ave_phi12'].std(), average_perchannel['ave_phi23'].std(), average_perchannel['ave_phi13'].std())

 angle per paire 125.64881455335913 95.61672351335247 -138.05264375147027 
 average mag per paire 179108.11071228757 146438.80396972594 230263.96580234275 
 total average of angle 27.73763143841377 
 STD per paire 0.2829257040522641 0.4643836003041056 0.27966520503374975


In [27]:
print(' Angle_per paire', np.average(average_perchannel['ave_phi12_opt'])*180/np.pi, np.average(average_perchannel['ave_phi23_opt'])*180/np.pi, np.average(average_perchannel['ave_phi13_opt'])*180/np.pi,
     '\n Average', (np.average(average_perchannel['ave_phi12_opt'])/3+ np.average(average_perchannel['ave_phi23_opt'])/3+ np.average(average_perchannel['ave_phi13_opt'])/3)*180/np.pi,
     '\n STD', average_perchannel['ave_phi12_opt'].std(), average_perchannel['ave_phi23_opt'].std(), average_perchannel['ave_phi13_opt'].std())

 Angle_per paire 125.47978290757813 95.96942017242111 -137.86897873818262 
 Average 27.860074780605537 
 STD 0.2824485790044729 0.464626553743889 0.281044869911218


In [122]:
# average_perchannel.unstack(level=0)[['ave_mag12']].plot(subplots=True, title="Average Phase per Channel", color= '#28156c', label='ave_mag12', legend=True)
# average_perchannel.unstack(level=0)[['ave_mag23']].plot(subplots=True, title="Average Phase per Channel", color= '#5f0a0a', label='ave_mag23', legend=True)
# average_perchannel.unstack(level=0)[['ave_mag13']].plot(subplots=True, title="Average Phase per Channel", color= '#ffaf00', label='ave_mag13', legend=True)

In [123]:
# average_perchannel.unstack(level=0)[['ave_phi12']].plot(subplots=True, title="Average Phase per Channel", color= '#28156c', label='ave_phi12', legend=True)
# average_perchannel.unstack(level=0)[['ave_phi23']].plot(subplots=True, title="Average Phase per Channel", color= '#5f0a0a', label='ave_phi23', legend=True)
# average_perchannel.unstack(level=0)[['ave_phi13']].plot(subplots=True, title="Average Phase per Channel", color= '#ffaf00', label='ave_phi13', legend=True)

# Importing the optimized corrected points

In [29]:
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative.csv'
path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_dasti_zero_optimized.csv'
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_45deg.csv'
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_30deg.csv'
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_dasti_for23.csv'
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_dasti.csv'
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_dasti_45deg.csv'
# path_coeff = 'C:/Users/pooneh/Documents/GitHub/AoA_IQsamples/AoA/DoA_Final/3_ANT/3ANT_AoA/datapoints_250cm/3ANT/zerocorrection_coeff_negative_dasti_30deg.csv'

df_corr = pd.read_csv(path_coeff)
df_corr.ex_phi12.mean()*180/np.pi

74.98450902022674

In [30]:
len(np.unique(df_corr.channel)), len(np.unique(average_perchannel.channel))

(36, 29)

In [31]:
np.unique(df_corr.channel)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35], dtype=int64)

In [32]:
np.unique(average_perchannel.channel)

array([ 1,  2,  3,  4,  5,  6,  8,  9, 11, 12, 13, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 25, 26, 27, 28, 30, 31, 33, 34, 35], dtype=int64)

In [33]:
channl = average_perchannel.channel
phi12_corr_ave = []
phi23_corr_ave = []
phi13_corr_ave = []
phi12_corr_ave_opt = []
phi23_corr_ave_opt = []
phi13_corr_ave_opt = []
# phi12_ave = []
for ch in channl:
    g = df_corr[df_corr.channel == ch]
    f = average_perchannel[average_perchannel.channel == ch]
#     print(ch, phi12_corr_ave)
    phi12_corr_ave.append(np.average(f.ave_phi12)- np.average(g.phi12_correction))
    phi23_corr_ave.append(np.average(f.ave_phi23)- np.average(g.phi23_correction))
    phi13_corr_ave.append(np.average(f.ave_phi13)- np.average(g.phi13_correction))
    phi12_corr_ave_opt.append(np.average(f.ave_phi12_opt)- np.average(g.phi12_correction))
    phi23_corr_ave_opt.append(np.average(f.ave_phi23_opt)- np.average(g.phi23_correction))
    phi13_corr_ave_opt.append(np.average(f.ave_phi13_opt)- np.average(g.phi13_correction))
#     phi12_ave.append(np.average(g.phi12_correction)*180/np.pi)
#     print(np.average(g.phi12_correction)*180/np.pi)


# print(len(average_perchannel), np.average(phi12_ave))
average_perchannel['corrected_phi12'] = phi12_corr_ave
average_perchannel['corrected_phi23'] = phi23_corr_ave
average_perchannel['corrected_phi13'] = phi13_corr_ave
average_perchannel['corrected_phi12_opt'] = phi12_corr_ave_opt
average_perchannel['corrected_phi23_opt'] = phi23_corr_ave_opt
average_perchannel['corrected_phi13_opt'] = phi13_corr_ave_opt
np.average(phi12_corr_ave)*180/np.pi, np.average(phi23_corr_ave)*180/np.pi, np.average(phi13_corr_ave)*180/np.pi,
np.max(phi12_corr_ave)*180/np.pi, np.max(phi23_corr_ave)*180/np.pi, np.max(phi13_corr_ave)*180/np.pi, np.min(phi13_corr_ave)*180/np.pi,

(96.72765058982698, 130.84402923793704, 157.5731706959464, 133.1270372425482)

In [34]:
len(phi12_corr_ave), len(df_corr), len(average_perchannel)

(66, 67, 66)

In [35]:
np.average(average_perchannel['corrected_phi12'])*180/np.pi, np.average(average_perchannel['corrected_phi13'])*180/np.pi, np.average(average_perchannel['corrected_phi23'])*180/np.pi

(78.98210718043927, 147.76049793830546, 87.22657761208626)

In [36]:
np.average(average_perchannel['corrected_phi12_opt'])*180/np.pi, np.average(average_perchannel['corrected_phi13_opt'])*180/np.pi, np.average(average_perchannel['corrected_phi23_opt'])*180/np.pi

(78.8130755346583, 147.9441629515931, 87.57927427115493)

In [37]:
np.average(average_perchannel['ave_phi12'])*180/np.pi, np.average(average_perchannel['ave_phi23'])*180/np.pi, np.average(average_perchannel['ave_phi13'])*180/np.pi

(125.64881455335913, 95.61672351335247, -138.05264375147027)

In [60]:
print('\n average phi', (np.average(average_perchannel['ave_phi12'])*180/np.pi+ np.average(average_perchannel['ave_phi23'])*180/np.pi+ np.average(average_perchannel['ave_phi13'])*90/np.pi)/3,
     '\n corrected phi_per channel', (np.average(average_perchannel['corrected_phi12'])*180/np.pi+ np.average(average_perchannel['corrected_phi13'])*90/np.pi+ np.average(average_perchannel['corrected_phi23'])*180/np.pi)/3,
)


 average phi 50.74640539699215 
 corrected phi_per channel 80.02964458722609


In [41]:
print(' average phi_opt', (np.average(average_perchannel['ave_phi12_opt'])*180/np.pi+ np.average(average_perchannel['ave_phi23_opt'])*180/np.pi+ np.average(average_perchannel['ave_phi13_opt'])*90/np.pi)/3,
     '\n corrected phi_per_opt channel', (np.average(average_perchannel['corrected_phi12_opt'])*180/np.pi+ np.average(average_perchannel['corrected_phi13_opt'])*90/np.pi+ np.average(average_perchannel['corrected_phi23_opt'])*180/np.pi)/3)

 average phi_opt 50.83823790363598 
 corrected phi_per_opt channel 80.12147709386993


In [61]:
print('\n average phi with no correction error', df_corr.ex_phi12.mean()*180/np.pi - (np.average(average_perchannel['ave_phi12'])*180/np.pi+ np.average(average_perchannel['ave_phi23'])*180/np.pi+ np.average(average_perchannel['ave_phi13'])*90/np.pi)/3,
     '\n corrected phi_per channel error', df_corr.ex_phi12.mean()*180/np.pi - (np.average(average_perchannel['corrected_phi12'])*180/np.pi+ np.average(average_perchannel['corrected_phi13'])*90/np.pi+ np.average(average_perchannel['corrected_phi23'])*180/np.pi)/3,
     )


 average phi with no correction error 24.238103623234586 
 corrected phi_per channel error -5.045135566999349


In [43]:
print('\n average phi_opt with no correction error', df_corr.ex_phi12.mean()*180/np.pi - (np.average(average_perchannel['ave_phi12_opt'])*180/np.pi+ np.average(average_perchannel['ave_phi23_opt'])*180/np.pi+ np.average(average_perchannel['ave_phi13_opt'])*90/np.pi)/3,
     '\n corrected phi_per_opt channel error', df_corr.ex_phi12.mean()*180/np.pi - (np.average(average_perchannel['corrected_phi12_opt'])*180/np.pi+ np.average(average_perchannel['corrected_phi13_opt'])*90/np.pi+ np.average(average_perchannel['corrected_phi23_opt'])*180/np.pi)/3,)


 average phi_opt with no correction error 24.14627111659076 
 corrected phi_per_opt channel error -5.136968073643189


In [44]:
df_corr.ex_phi12.mean()*180/np.pi

74.98450902022674

calculate the standard deviation of corrected signal, per antenna! (averaging the valus (phi12/3, phi23/3) and phi13/6)

In [45]:
print('standard deviation of corrected phi', average_perchannel['corrected_phi12'].std()*60/np.pi, average_perchannel['corrected_phi23'].std()*60/np.pi, average_perchannel['corrected_phi13'].std()*30/np.pi)

standard deviation of corrected phi 3.0300167523045367 8.922659510042124 0.8957651398601303


In [46]:
print('standard deviation of corrected phi', average_perchannel['corrected_phi12_opt'].std()*60/np.pi, average_perchannel['corrected_phi23_opt'].std()*60/np.pi, average_perchannel['corrected_phi13_opt'].std()*30/np.pi)

standard deviation of corrected phi 3.0404065483938902 8.935645250561409 0.9042554609821292


As we can see here, the corrected valus are not equal to the expected value arround 75 degrees. As a result, we still need extra correction per channel for our system. meaning that, we can find an optimom correction for each channel. 

In [34]:
# average_perchannel.unstack(level=0)[['ave_phi12']].plot(subplots=True, title="Average Phase per Channel", color= '#28156c', label='ave_phi12', legend=True)
# average_perchannel.unstack(level=0)[['corrected_phi12']].plot(subplots=True, title="Average Phase per Channel", color= '#5f0a0a', label='corrected_phi12', legend=True)
# df_corr.unstack(level=0)[['phi12_correction']].plot(subplots=True, title="Average Phase per Channel", color= '#ffaf00', label='expected_phi12_value', legend=True)

In [35]:
# average_perchannel.unstack(level=0)[['ave_phi23']].plot(subplots=True, title="Average Phase per Channel", color= '#28156c', label='ave_phi23', legend=True)
# average_perchannel.unstack(level=0)[['corrected_phi23']].plot(subplots=True, title="Average Phase per Channel", color= '#5f0a0a', label='corrected_phi23', legend=True)
# df_corr.unstack(level=0)[['phi23_correction']].plot(subplots=True, title="Average Phase per Channel", color= '#ffaf00', label='expected_phi23_value', legend=True)

In [36]:
# average_perchannel.unstack(level=0)[['ave_phi13']].plot(subplots=True, title="Average Phase per Channel", color= '#28156c', label='ave_phi13', legend=True)
# average_perchannel.unstack(level=0)[['corrected_phi13']].plot(subplots=True, title="Average Phase per Channel", color= '#5f0a0a', label='corrected_phi13', legend=True)
# df_corr.unstack(level=0)[['phi13_correction']].plot(subplots=True, title="Average Phase per Channel", color= '#ffaf00', label='expected_phi13_value', legend=True)

Now calculate the actual angle! 

In [47]:
a12 = np.average(average_perchannel['corrected_phi12']) 
a23 = np.average(average_perchannel['corrected_phi23'])
a13 = np.average(average_perchannel['corrected_phi13'])/2 
landa = 2.9e8/(average_perchannel.omega*1e6)
d12 = 3.5e-2
theta12 = np.arccos(a12*landa/(2*np.pi*d12)) - np.deg2rad(45)
theta23 = np.arccos(a23*landa/(2*2*np.pi*d12)) - np.deg2rad(45)
theta13 = np.arccos(a13*landa/(2*2*np.pi*d12)) - np.deg2rad(45)

In [48]:
print(np.average(theta12*180/np.pi), np.average(theta23*180/np.pi), np.average(theta13*180/np.pi),
      (np.average(theta12*180/np.pi)+ np.average(theta23*180/np.pi)+ np.average(theta13*180/np.pi))/3)

-3.1280530271933626 20.721762628276483 24.619349724545664 14.071019775209594


# Optimization
non-linear least squre curve fitting

# kalman_filter

The input is:  
x = corrected_phi  
v = omegat  
R = covariance2d(error_obs_x, error_obs_v), measurment error  
P = covariance2d(error_est_x, error_est_v), est_error  
Obviously the Kalman filter is not efficient yet! find a better filter. Something that can use a ground truth somewhere.

In [49]:
sdata = average_perchannel
len(sdata)

66

In [62]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

In [63]:
t = 2.5e-8
error_est_x = -0.2272
error_est_x23 = -0.3128
error_est_x13 = -0.178
print(error_est_x, error_est_x23, error_est_x13)
error_est_v = 5;
v = 2404
def prediction2d(x, v, t):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    X_prime = A.dot(X) 
    return X_prime

def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))

# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
P23 = covariance2d(error_est_x23, error_est_v)
P13 = covariance2d(error_est_x13, error_est_v)

A = np.array([[1, t],
              [0, 1]])

error_obs_v = int(sdata['omega'].std())
# error_obs_x = sdata['corrected_phi12'].std()
# error_obs_x23 = sdata['corrected_phi23'].std()
# error_obs_x13 = sdata['corrected_phi12'].std()

error_obs_x = 0.328
error_obs_x23 = 0.467
error_obs_x13 = 0.2695

print(error_obs_x, error_obs_x23, error_obs_x13)

# x_observations = sdata['corrected_phi12']
# x23_observations = sdata['corrected_phi23']
# x13_observations = sdata['corrected_phi13']
x_observations = sdata['corrected_phi12_opt']
x23_observations = sdata['corrected_phi23_opt']
x13_observations = sdata['corrected_phi13_opt']

v_observations = sdata['omega']
v = sdata['omega'].mean()
z = np.c_[x_observations, v_observations]
z23 = np.c_[x23_observations, v_observations]
z13 = np.c_[x13_observations, v_observations]

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
X23 = np.array([[z23[0][0]],
              [v]])
X13 = np.array([[z13[0][0]],
              [v]])

n = len(z[0])

-0.2272 -0.3128 -0.178
0.328 0.467 0.2695


In [65]:
A

array([[1.0e+00, 2.5e-08],
       [0.0e+00, 1.0e+00]])

In [64]:
results = []
for i in range(len(z)):
    for data in z[1:]:
        X = prediction2d(X[0][0], X[1][0], t)
        # To simplify the problem, professor
        # set off-diagonal terms to 0.
        P = np.diag(np.diag(A.dot(P).dot(A.T)))

        # Calculating the Kalman Gain
        H = np.identity(n)
        R = covariance2d(error_obs_x, error_obs_v)
        S = H.dot(P).dot(H.T) + R
        from numpy.linalg import inv
        K = P.dot(H).dot(inv(S))
        del inv

        # Reshape the new data into the measurement space.
        Y = H.dot(data).reshape(n, -1)

        # Update the State Matrix
        # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
        X = X + K.dot(Y - H.dot(X))
        
        # Update Process Covariance Matrix
        P = (np.identity(len(K)) - K.dot(H)).dot(P)
    results.append(X)


In [52]:

print("Kalman Filter State Matrix:\n", X,'\n The expected_delta_phi is:',X[0]*180/np.pi,'\n it was:', a12*180/np.pi, 
      '\n and it should be:',df_corr.ex_phi12.mean()*180/np.pi, '\n error befor:', abs(df_corr.ex_phi12.mean()-a12)*180/np.pi,
     '\n error after:', abs(df_corr.ex_phi12.mean()*180/np.pi-X[0]*180/np.pi))

Kalman Filter State Matrix:
 [[1.50705569e+00]
 [2.44099045e+03]] 
 The expected_delta_phi is: [86.34793068] 
 it was: 78.98210718043927 
 and it should be: 74.98450902022674 
 error befor: 3.9975981602125277 
 error after: [11.36342166]


In [53]:
results = []
for i in range(len(z23)):
    for data in z23[1:]:
        X23 = prediction2d(X23[0][0], X23[1][0], t)
        # To simplify the problem, professor
        # set off-diagonal terms to 0.
        P23 = np.diag(np.diag(A.dot(P23).dot(A.T)))

        # Calculating the Kalman Gain
        H = np.identity(n)
        R = covariance2d(error_obs_x23, error_obs_v)
        S = H.dot(P23).dot(H.T) + R
        from numpy.linalg import inv
        K = P23.dot(H).dot(inv(S))
        del inv

        # Reshape the new data into the measurement space.
        Y = H.dot(data).reshape(n, -1)

        # Update the State Matrix
        # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
        X23 = X23 + K.dot(Y - H.dot(X23))
        
        # Update Process Covariance Matrix
        P = (np.identity(len(K)) - K.dot(H)).dot(P)
    results.append(X23)

In [54]:
print("Kalman Filter State Matrix:\n", X,'\n The expected_delta_phi with kalman_filter is:',X23[0]*180/np.pi,
      '\n expected_delta_phi was:', a23*180/np.pi, 
      '\n and it should be:',df_corr.ex_phi12.mean()*180/np.pi, '\n error befor:', abs(df_corr.ex_phi12.mean()-a23)*180/np.pi,
     '\n error after:', abs(df_corr.ex_phi12.mean()*180/np.pi-X23[0]*180/np.pi))

Kalman Filter State Matrix:
 [[1.50705569e+00]
 [2.44099045e+03]] 
 The expected_delta_phi with kalman_filter is: [46.19500766] 
 expected_delta_phi was: 87.22657761208626 
 and it should be: 74.98450902022674 
 error befor: 12.242068591859509 
 error after: [28.78950136]


In [55]:
results = []
for i in range(len(z13)):
    for data in z13[1:]:
        X13 = prediction2d(X13[0][0], X13[1][0], t)
        # To simplify the problem, professor
        # set off-diagonal terms to 0.
        P13 = np.diag(np.diag(A.dot(P13).dot(A.T)))

        # Calculating the Kalman Gain
        H = np.identity(n)
        R = covariance2d(error_obs_x13, error_obs_v)
        S = H.dot(P13).dot(H.T) + R
        from numpy.linalg import inv
        K = P13.dot(H).dot(inv(S))
        del inv

        # Reshape the new data into the measurement space.
        Y = H.dot(data).reshape(n, -1)

        # Update the State Matrix
        # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
        X13 = X13 + K.dot(Y - H.dot(X13))
        
        # Update Process Covariance Matrix
        P = (np.identity(len(K)) - K.dot(H)).dot(P)
    results.append(X13)

In [56]:
print("Kalman Filter State Matrix:\n", X13,'\n The expected_delta_phi after kalman filter is:',X13[0]*180/np.pi,
      '\n expected_delta_phi was:', a13*360/np.pi, 
      '\n and it should be:',df_corr.ex_phi12.mean()*360/np.pi, 
      '\n error befor:', abs(df_corr.ex_phi12.mean()-a13)*360/np.pi,
     '\n error after:', abs(df_corr.ex_phi12.mean()*2-X13[0])*180/np.pi)

Kalman Filter State Matrix:
 [[   2.45907274]
 [2426.19336006]] 
 The expected_delta_phi after kalman filter is: [140.89448953] 
 expected_delta_phi was: 147.76049793830546 
 and it should be: 149.96901804045348 
 error befor: 2.2085201021480327 
 error after: [9.07452851]


In [57]:
print(df_corr.ex_phi12.mean()*180/np.pi - (X[0]+X23[0]+X13[0])/4*180/np.pi)

[6.62515205]


In [58]:
 (X[0]+X23[0]+X13[0])/4*180/np.pi

array([68.35935697])

In [59]:
df_corr.ex_phi12.mean()*180/np.pi - (X[0]+X23[0]+X13[0])/3*180/np.pi

array([-16.16130027])